In [1]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**GENERATE CSV**

In [0]:
import os, csv

f=open("/content/drive/My Drive/sarseg/Data/complete_dataset/crop/noisy_img.csv",'w+')
w=csv.writer(f)
for path, dirs, files in os.walk("/content/drive/My Drive/sarseg/Data/complete_dataset/crop/noisy images"):
    for filename in files:
      w.writerow([filename])

In [0]:
import cv2
import numpy as np

from keras.preprocessing.image import img_to_array


def category_label(labels, dims, n_labels):
    x = np.zeros([dims[0], dims[1], n_labels])
    #print(labels)
    for i in range(dims[0]):
        for j in range(dims[1]):
          x[i, j, (int)(labels[i][j]/255)] = 1
    x = x.reshape(dims[0] * dims[1], n_labels)
    return x


def data_gen_small(img_dir, mask_dir, lists, batch_size, dims, n_labels):
    while True:
        ix = np.random.choice(np.arange(len(lists)), batch_size)
        imgs = []
        labels = []
        for i in ix:
            # images
            img_path = img_dir + lists.iloc[i, 0]
#            print( img_path )
            original_img = cv2.imread(img_path)[:, :, ::-1]
            resized_img = cv2.resize(original_img, (dims[0], dims[1]))
            array_img = img_to_array(resized_img)
#            print(array_img.size)
            imgs.append(array_img)
            # masks
            original_mask = cv2.imread(mask_dir + lists.iloc[i, 0])
            resized_mask = cv2.resize(original_mask, (dims[0], dims[1]))
#            print(resized_mask.shape)
            array_mask = category_label(resized_mask[:, :, 0], dims, n_labels)
            labels.append(array_mask)
        imgs = np.array(imgs)
        labels = np.array(labels)
        yield imgs, labels

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarn

In [0]:
!pip install tensorflow==1.8.0
!pip install Keras==2.2.4

     |████████████████████████████████| 49.1MB 85kB/s 
     |████████████████████████████████| 3.1MB 45.5MB/s 
     |████████████████████████████████| 890kB 38.8MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=f1a625341fdb2f6e59f57115aad02d2da8b6752bbe1f3f300723fd7c72f3a461
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.4
    Uninstalling bleach-3.1.4:
      Successfully uninstalled bleach-3.1.4
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow 2.2.0rc3
    Uninstalling tensorflow-2.2.0rc3:
      Successfully uninstalled tensorflow-2.2.0rc3


     |████████████████████████████████| 317kB 3.4MB/s 
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [0]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
  c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(x)
  c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(c)
  p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
  return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
  us = keras.layers.UpSampling2D((2, 2))(x)
  concat = keras.layers.Concatenate()([us, skip])
  c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(concat)
  c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(c)
  return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
  c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(x)
  c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(c)
  return c
  
def UNet():
  f = [32, 64, 128, 256, 512]
  inputs = keras.layers.Input((512, 512, 3))
  p0 = inputs
  c1, p1 = down_block(p0, f[0])
  c2, p2 = down_block(p1, f[1])
  c3, p3 = down_block(p2, f[2])
  c4, p4 = down_block(p3, f[3])

  bn = bottleneck(p4, f[4])
  u1 = up_block(bn, c4, f[3])
  u2 = up_block(u1, c3, f[2])
  u3 = up_block(u2, c2, f[1])
  u4 = up_block(u3, c1, f[0])
  outputs = keras.layers.Conv2D(2, (1, 1), padding="same", activation="sigmoid")(u4)
  outputs = Reshape(
            (512*512, 2),
            input_shape=(512, 512, 2))(outputs)
  model = keras.models.Model(inputs, outputs)
  return model

In [0]:
model = UNet()
model.summary()

NameError: ignored

In [0]:
import os
import sys
import random
import warnings
import pickle
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import keras
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import cv2
import tensorflow as tf

Generator.py

In [0]:
import keras
import pandas as pd

from keras.models import load_model

def load(model_name):
  model_test = load_model("/content/drive/My Drive/sarseg/SAR dataset"+model_name+".h5")
  model_test.load_weights("/content/drive/My Drive/sarseg/SAR dataset"+model_name+".hdf5")
  print('model loaded')
  return model_test

model = "SAR_40_epochs"
model_test = load(model)

model loaded


In [0]:
def train_resume(model_name):    
    train_list = pd.read_csv("/content/drive/My Drive/sarseg/Data/complete_dataset/crop/noisy_img.csv", header=None)
    val_list = pd.read_csv("/content/drive/My Drive/sarseg/Data/complete_dataset/crop/noisy_img.csv", header=None)

    # set the necessary directories
    trainimg_dir = "/content/drive/My Drive/sarseg/Data/complete_dataset/crop/noisy images/"
    trainmsk_dir = "/content/drive/My Drive/sarseg/Data/complete_dataset/crop/gt/"
    valimg_dir = "/content/drive/My Drive/sarseg/Data/complete_dataset/crop/noisy images/"
    valmsk_dir = "/content/drive/My Drive/sarseg/Data/complete_dataset/crop/gt/"

    train_gen = data_gen_small(trainimg_dir, trainmsk_dir,
            train_list, 8,
            [512, 512], 2)
    val_gen = data_gen_small(valimg_dir, valmsk_dir,
            val_list, 8,
            [512, 512], 2)
    model_test.fit_generator(train_gen, steps_per_epoch=8,
             epochs=10, validation_data=val_gen,
             validation_steps=8)
    model_test.save_weights("/content/drive/My Drive/sarseg/SAR dataset"+model_name +".hdf5")
    model_test.save("/content/drive/My Drive/sarseg/SAR dataset"+model_name +".h5")
    print("save weight done..")

new_model = "SAR_40_epochs"
train_resume(new_model)

In [0]:
model_name = "SAR_21_epochs"
model_test.save_weights("/content/drive/My Drive/sarseg/SAR dataset"+model_name +".hdf5")
model_test.save("/content/drive/My Drive/sarseg/SAR dataset"+model_name +".h5")
print("save weight done..")